## Read the PDFs and split them into chunks

In [4]:
import fitz  # PyMuPDF
import re
import ssl
from pathlib import Path

def load_pdfs(data_path="data/"):
    documents = []
    for pdf_path in Path(data_path).glob("*.pdf"):
        with fitz.open(pdf_path) as doc:
            text = ""
            for page in doc:
                text += page.get_text()
        documents.append({"filename": pdf_path.name, "content": text})
    return documents

def chunk_text(text, max_tokens=300):
    import re
    sentences = re.split(r'(?<=[.!?]) +', text)
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk.split()) + len(sentence.split()) < max_tokens:
            current_chunk += " " + sentence
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks





## Generate embeddings

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from sentence_transformers import SentenceTransformer
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


No sentence-transformers model found with name sentence-transformers/all-MiniLM-L6-v2. Creating a new one with mean pooling.


SSLError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/adapter_config.json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1028)')))"), '(Request ID: c980fb9e-2cb0-4a30-9aaf-d7e6b733c6ad)')